# 共同研究者を管理する

GRDMプロジェクトの共同研究者の追加や削除、権限の管理を行う。

## 共同編集者の表示
GRDMに登録されている共同研究者を表示します。

・アクセストークン
・プロジェクトID（暫定）
・表示（ボタン）

共同研究者の編集はGRDMのプロジェクトのメニューにあるメンバーから行ってください。
※【注意】メンバーの権限が「読込み/書込み」「管理者」のいずれかに設定されているメンバーでないとリサーチフローは操作できません。

In [ ]:
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.common.manage_collaborators import CollaboratorManager
CollaboratorManager(working_path=os.path.abspath('__file__')).generateFormScetion()

## サブフローメニューへ

サブフローメニューへアクセスするボタンを表示します

In [ ]:
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.common.manage_collaborators import CollaboratorManager
CollaboratorManager(working_path=os.path.abspath('__file__')).return_subflow_menu()